In [1]:
import os
import tvm
import nnvm
import topi

import nnvm.symbol as sb
from tvm.contrib import graph_runtime

import numpy as np
import tensorflow as tf

EXAMPLE = 1
target, target_host = 'llvm', 'llvm'
ctx = tvm.cpu(0)



In [2]:
import xdnn

libxilinxopencl.so: cannot open shared object file: No such file or directory


In [3]:
platform = 'alveo-u200'

config = {
    'platform': platform,
    'xclbin': "../overlaybins/" + platform + "/overlay_3.xclbin",
    'memory': 5,
    'dsp': 56,
    'netcfg': "work/tvm_fpga.cmds.json",
    #'fromtensorflow': False,
    'weightsdir_name': "weights", 
    #datadir': "work/weights_data",
    'quantizecfg': "", # Empty for now to test if quantization is needed in this simple example
    'pngfile': "graph.png",
    'verbose': True
}

xdnn.xdnn_frontend.init(
    platform=config['platform'],
    xclbin=config['xclbin'],
    memory=config['memory'],
    dsp=config['dsp'],
    netcfg=config['netcfg'],
    weightsdir_name=config['weightsdir_name'],
    pngfile=config['pngfile'],
    verbose=config['verbose']
)

if EXAMPLE == 1:
    input_shape = (1,1,4,4)
    input_type = 'float32'
    output_shape = (1,1,2,2)
    
    data = np.reshape(np.array([[1,1,0,4],[5,1,0,8],
                                [3,5,1,0],[1,9,3,4]]), input_shape)
    x = sb.Variable("x")
    z = sb.max_pool2d(x, pool_size=[2,2], strides=[2,2])
    graph = nnvm.graph.create(z)
    print(graph.index.nodes)
    
    params = {}
    shape_dict = { 'x': input_shape }
    dtype_dict = { 'x': input_type }
    graph, lib, params = nnvm.compiler.build(
                graph, target, shape_dict, dtype_dict,
                params=params, target_host=target_host)
    
    print(graph.index.nodes)
    print(lib)
    print(params)
    
    m = graph_runtime.create(graph, lib, ctx)
    m.set_input('x', tvm.nd.array(data.astype(input_type)))
    
    m.run()
    
    tvm_output = m.get_output(0, tvm.nd.empty(((1, 1, 2, 2)), 'float32'))
    
    print(tvm_output)

networkfile None
memory 5
dsp 56
generatefile temp_fpga_code.cmds
fromtensorflow False
weights weights
pngfile graph.png
verbose True
Namespace(anew=None, approximate=False, banditpre=None, barrier=False, bridges=None, bytesperpixels=2, concatstrategy=None, conv_1x1_s2=False, cpulayermustgo=False, ddr=256, dedicateddsp=None, dsp=56, fc=False, finalnode=None, forceweights=None, fromtensorflow=False, frontendonly=False, generatefile='temp_fpga_code.cmds', godreplication=None, godtiling=None, initialnode=None, lasttensorbyname=None, leavescalealone=False, loadpickle=None, manasadebugmode=False, manualbatch=False, manualdeconv=False, memory=5, mixmemorystrategy=False, networkfile='None', nodynamicscaling=False, noreplication=False, notcaffeanew=False, parallelism=False, parallelismgraphalgorithm='tfs', parallelismstrategy=None, parallelread=None, pipelineconvmaxpool=False, placeholdershape=None, pngfile='graph.png', poolingaround=False, quant_cfgfile=None, rankdir='BT', savepickle=None, sc

AttributeError: module 'xfdnn.rt.xdnn' has no attribute 'createHandle'